Filename identifiers

Modality (01 = full-AV, 02 = video-only, 03 = audio-only). Vocal channel (01 = speech, 02 = song). Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised). Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion. Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door"). Repetition (01 = 1st repetition, 02 = 2nd repetition). Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

Questions pour mardi :  
Pourquoi mon accuracy score change t-il?  
Comment améliorer les paramètres du model? 
Utiliser un autre model?  
Est-ce que je peux aavoir le résultat des échantillons d'émotions sur la même ligne -> pour l'exploiter dans la présentation et le read-me?

In [43]:
import librosa as lr
import librosa.display
import librosa 
import os
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt

import pickle

import seaborn as sns

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import warnings

warnings.simplefilter("ignore")
pd.options.display.max_columns = None

In [ ]:
# CREATE DIRECTORY OF AUDIO FILES 
audio = "/Users/diebl/lab/FinalProject__RecognitionSpeechEmotion/audio_speech_actors_01-24/"
actor_folders = os.listdir(audio) #list files in audio directory
actor_folders.sort() 
actor_folders[0:5]

In [ ]:
# EXTRACT EMOTION NUMBER, ACTOR AND GENDER LABEL
emotion = []
gender = []
actor = []
file_path = []
for i in actor_folders:
    filename = os.listdir(audio + i) #iterate over Actor folders
    for f in filename: # go through files in Actor folder
        part = f.split('.')[0].split('-')
        emotion.append(int(part[2]))
        actor.append(int(part[6]))
        bg = int(part[6])
        if bg%2 == 0:
            bg = "female"
        else:
            bg = "male"
        gender.append(bg)
        file_path.append(audio + i + '/' + f)

In [ ]:
# PUT EXTRACTED LABELS WITH FILEPATH INTO DATAFRAME
audio_df = pd.DataFrame(emotion)
audio_df = audio_df.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
audio_df = pd.concat([pd.DataFrame(gender),audio_df,pd.DataFrame(actor)],axis=1)
audio_df.columns = ['gender','emotion','actor']
audio_df = pd.concat([audio_df,pd.DataFrame(file_path, columns = ['path'])],axis=1)
audio_df.head()

In [ ]:
# ENSURE GENDER,EMOTION, AND ACTOR COLUMN VALUES ARE CORRECT
pd.set_option('display.max_colwidth', -1)
#Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
audio_df.sample(10)

In [ ]:
# LOOK AT DISTRIBUTION OF CLASSES
audio_df.emotion.value_counts().plot(kind='bar')

In [2]:
audio_df = pd.read_csv('audio_df.csv')

In [ ]:
# ITERATE OVER ALL AUDIO FILES AND EXTRACT LOG MEL SPECTROGRAM MEAN VALUES INTO DF FOR MODELING 
df = pd.DataFrame(columns=['mel_spectrogram'])

counter=0

for index,path in enumerate(audio_df.path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=3,sr=44100,offset=0.5)
    
    #get the mel-scaled spectrogram (transform both the y-axis (frequency) to log scale, 
    #and the “color” axis (amplitude) to Decibels, which is kinda the log scale of amplitudes.)
    spectrogram = librosa.feature.melspectrogram(y=X, sr=sample_rate, n_mels=128,fmax=8000) 
    db_spec = librosa.power_to_db(spectrogram)
    #temporally average spectrogram
    log_spectrogram = np.mean(db_spec, axis = 0)
        
    df.loc[counter] = [log_spectrogram]
    counter=counter+1   


In [ ]:
df.head()
#df.info()

In [ ]:
# TURN ARRAY INTO LIST AND JOIN WITH AUDIO_DF TO GET CORRESPONDING EMOTION LABELS
df_final = pd.concat([audio_df,pd.DataFrame(df['mel_spectrogram'].values.tolist())],axis=1)
df_final = df_final.fillna(0)

In [ ]:
df_final.info()

In [ ]:
df_final.head()

In [ ]:
# DROP PATH COLUMN FOR MODELING
df_final.drop(columns='path',inplace=True)

In [ ]:
df_final.to_csv('mel_24_8.csv',index = False)

In [3]:
df_final = pd.read_csv('mel_24_8.csv')
df_final.head()

,gender,emotion,actor,0,1,2,3,4,5,6,...,249,250,251,252,253,254,255,256,257,258
0,male,neutral,1,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,male,neutral,1,-75.335518,-75.445320,-75.554031,-75.203949,-75.230530,-75.319374,-75.653793,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,male,neutral,1,-75.150711,-75.150711,-75.150711,-75.150711,-75.150711,-75.150711,-75.150711,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,male,neutral,1,-75.268448,-75.268448,-75.268448,-75.268448,-75.268448,-75.268448,-75.268448,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,male,calm,1,-80.147377,-80.147377,-80.147377,-80.147377,-80.147377,-80.147377,-80.147377,...,-80.147377,-80.147377,-80.121956,-79.998009,-80.119255,-80.147377,-80.130417,-80.014702,-80.147377,-80.147377


## MODELING

In [4]:
# drop the target value
X = df_final.drop('emotion', axis=1)
y = df_final['emotion']


In [5]:
#encoding gender feature

X= pd.get_dummies(X)
X.head()

,actor,0,1,2,3,4,5,6,7,8,...,251,252,253,254,255,256,257,258,gender_female,gender_male
0,1,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,-76.384773,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1
1,1,-75.335518,-75.445320,-75.554031,-75.203949,-75.230530,-75.319374,-75.653793,-75.720039,-75.720039,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1
2,1,-75.150711,-75.150711,-75.150711,-75.150711,-75.150711,-75.150711,-75.150711,-75.139587,-75.072227,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1
3,1,-75.268448,-75.268448,-75.268448,-75.268448,-75.268448,-75.268448,-75.268448,-75.268448,-75.268448,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1
4,1,-80.147377,-80.147377,-80.147377,-80.147377,-80.147377,-80.147377,-80.147377,-80.147377,-80.147377,...,-80.121956,-79.998009,-80.119255,-80.147377,-80.130417,-80.014702,-80.147377,-80.147377,0,1


In [ ]:
X.info()

In [ ]:
X.describe()

In [6]:
X1 = X.copy()

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Faire une fonction sans standardization et une avec

In [8]:
def MLPClassifierModeling(X_train, X_test, y_train, y_test):
    model = MLPClassifier(hidden_layer_sizes=(200,), learning_rate='adaptive', max_iter=400)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
    print("Accuracy of the Recognizer is: {:.1f}%".format(accuracy*100))
    emotions = ['neutral','calm', 'happy', 'sad', 'angry', 'fear', 'disgust', 'surprise']
    print(classification_report(y_test, y_pred, labels=['neutral','calm', 'happy', 'sad', 'angry', 'fear', 'disgust', 'surprise']))
    
    
    print(f"y_test[10:30]={y_test[10:30]}")
    print(f"y_pred[10:30]={y_pred[10:30]}")

In [9]:
MLPClassifierModeling(X_train, X_test, y_train, y_test)

Accuracy of the Recognizer is: 25.0%
              precision    recall  f1-score   support

     neutral       0.00      0.00      0.00        20
        calm       0.31      0.18      0.23        44
       happy       0.19      0.12      0.15        34
         sad       0.19      0.67      0.29        39
       angry       0.68      0.31      0.43        42
        fear       0.20      0.03      0.05        32
     disgust       0.25      0.59      0.36        32
    surprise       1.00      0.02      0.04        45

    accuracy                           0.25       288
   macro avg       0.35      0.24      0.19       288
weighted avg       0.40      0.25      0.21       288

y_test[10:30]=589      disgust
115     surprise
1223         sad
940         fear
141          sad
1305     disgust
695        angry
756         fear
306         calm
350      disgust
1192    surprise
1023     neutral
1140     neutral
620          sad
819         fear
746          sad
654     surprise
415     s

Analysis of the accuracy :  
AS :27,8%

Then AS : 25%



## Improving the model

In [ ]:
sns.distplot(df_final[94], bins=40)
plt.show()




**with standardization**

In [10]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.2, random_state=42)

In [11]:
#standardization

transformer = StandardScaler()
transformer.fit(X1_train)

# saving in a pickle
with open('std_transformer1.pickle', 'wb') as file:
    pickle.dump(transformer, file)
    
# loading from a pickle  
with open('std_transformer1.pickle', 'rb') as file:
    loaded_standardized = pickle.load(file)

X1_train = loaded_standardized.transform(X1_train)
X1_test = loaded_standardized.transform(X1_test)

In [12]:
MLPClassifierModeling(X1_train, X1_test, y1_train, y1_test)

Accuracy of the Recognizer is: 46.9%
              precision    recall  f1-score   support

     neutral       0.23      0.30      0.26        20
        calm       0.68      0.64      0.66        44
       happy       0.29      0.47      0.36        34
         sad       0.46      0.41      0.43        39
       angry       0.67      0.52      0.59        42
        fear       0.39      0.44      0.41        32
     disgust       0.47      0.47      0.47        32
    surprise       0.60      0.40      0.48        45

    accuracy                           0.47       288
   macro avg       0.47      0.46      0.46       288
weighted avg       0.50      0.47      0.48       288

y_test[10:30]=589      disgust
115     surprise
1223         sad
940         fear
141          sad
1305     disgust
695        angry
756         fear
306         calm
350      disgust
1192    surprise
1023     neutral
1140     neutral
620          sad
819         fear
746          sad
654     surprise
415     s

We have increased the accuracy of our model. AS = 45,1% Then 46,9%

## With other features in addition to mel spectogramm

In [ ]:
# ITERATE OVER ALL AUDIO FILES AND EXTRACT MEL SPECTROGRAM MEAN VALUES INTO DF FOR MODELING 
df1 = pd.DataFrame(columns=['mel_spectro' ])

counter=0

for index,path in enumerate(audio_df.path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=3,sr=44100,offset=0.5)
    
    #get the mel-scaled spectrogram (transform both the y-axis (frequency) to log scale, 
    
    #spectrogram = librosa.feature.melspectrogram(y=X, sr=sample_rate, n_mels=128,fmax=8000)
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
  
        
    df1.loc[counter] = [mel]
    counter=counter+1   
    

In [ ]:
df1.info()

In [ ]:
# ITERATE OVER ALL AUDIO FILES AND EXTRACT CHROMA  MEAN VALUES INTO DF FOR MODELING 
df2 = pd.DataFrame(columns=['chroma' ])
stft=np.abs(librosa.stft(X))
counter=0

for index,path in enumerate(audio_df.path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=3,sr=44100,offset=0.5)
    
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
  
        
    df2.loc[counter] = [chroma]
    counter=counter+1   
    

In [ ]:
df2.info()

In [ ]:
# ITERATE OVER ALL AUDIO FILES AND EXTRACT MFCCS MEAN VALUES INTO DF FOR MODELING 
df3 = pd.DataFrame(columns=['mfccs' ])

counter=0

for index,path in enumerate(audio_df.path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=3,sr=44100,offset=0.5)
    
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
  
        
    df3.loc[counter] = [mfccs]
    counter=counter+1   
    

In [ ]:
df3.info()

In [ ]:
# TURN ARRAY INTO LIST AND JOIN WITH AUDIO_DF TO GET CORRESPONDING EMOTION LABELS
df_features = pd.concat([audio_df,pd.DataFrame(df1['mel_spectro'].values.tolist()),pd.DataFrame(df2['chroma'].values.tolist()),pd.DataFrame(df3['mfccs'].values.tolist())],axis=1)
d_features= df_features.fillna(0)

In [ ]:
df_features.head()

In [ ]:
# DROP PATH COLUMN FOR MODELING
df_features.drop(columns='path',inplace=True)

In [ ]:
df_features.to_csv('mel_chroma_mfccs_24_8.csv',index = False)

In [13]:
df_features = pd.read_csv('mel_chroma_mfccs_24_8.csv')
df_features.head()

,gender,emotion,actor,0,1,2,3,4,5,6,...,30.1,31.1,32.1,33.1,34.1,35.1,36.1,37.1,38.1,39.1
0,male,neutral,1,0.000011,0.001976,0.015901,0.032733,0.021538,0.008141,0.028234,...,-0.320704,-3.949646,-1.170431,-0.233570,-1.211523,0.016120,-1.144806,-2.061026,-1.967518,-1.471878
1,male,neutral,1,0.000014,0.001849,0.014971,0.047551,0.035372,0.006433,0.024542,...,-1.516942,-4.404281,-0.814472,0.109777,-2.077316,-0.285888,-0.560736,-2.559335,-1.591917,-1.465123
2,male,neutral,1,0.000017,0.001908,0.017518,0.046132,0.024488,0.009170,0.032045,...,-0.307369,-3.435000,-1.965031,-0.523855,-1.918070,-0.327916,-1.371087,-2.072444,-2.138656,-1.685442
3,male,neutral,1,0.000014,0.003084,0.023976,0.059002,0.025616,0.006316,0.024183,...,-0.903333,-3.661112,-1.938478,-0.868744,-1.534840,0.441906,-1.456406,-2.480440,-2.130476,-2.133232
4,male,calm,1,0.000012,0.000789,0.008584,0.023643,0.024978,0.006251,0.016150,...,-1.139909,-4.333735,-0.504630,-0.390943,-2.260401,0.173424,-1.041390,-1.922247,-0.495608,-1.814612


## Modeling

In [14]:
# drop the target value
X = df_features.drop('emotion', axis=1)
y = df_features['emotion']

In [15]:
#encoding gender feature

X= pd.get_dummies(X)
X.head()

,actor,0,1,2,3,4,5,6,7,8,...,32.1,33.1,34.1,35.1,36.1,37.1,38.1,39.1,gender_female,gender_male
0,1,0.000011,0.001976,0.015901,0.032733,0.021538,0.008141,0.028234,0.022807,0.027077,...,-1.170431,-0.233570,-1.211523,0.016120,-1.144806,-2.061026,-1.967518,-1.471878,0,1
1,1,0.000014,0.001849,0.014971,0.047551,0.035372,0.006433,0.024542,0.038484,0.046205,...,-0.814472,0.109777,-2.077316,-0.285888,-0.560736,-2.559335,-1.591917,-1.465123,0,1
2,1,0.000017,0.001908,0.017518,0.046132,0.024488,0.009170,0.032045,0.041563,0.050883,...,-1.965031,-0.523855,-1.918070,-0.327916,-1.371087,-2.072444,-2.138656,-1.685442,0,1
3,1,0.000014,0.003084,0.023976,0.059002,0.025616,0.006316,0.024183,0.030126,0.023210,...,-1.938478,-0.868744,-1.534840,0.441906,-1.456406,-2.480440,-2.130476,-2.133232,0,1
4,1,0.000012,0.000789,0.008584,0.023643,0.024978,0.006251,0.016150,0.017182,0.019608,...,-0.504630,-0.390943,-2.260401,0.173424,-1.041390,-1.922247,-0.495608,-1.814612,0,1


In [ ]:
X.describe()

In [16]:
X1 = X.copy()

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
MLPClassifierModeling(X_train, X_test, y_train, y_test)

Accuracy of the Recognizer is: 50.3%
              precision    recall  f1-score   support

     neutral       0.21      0.85      0.34        20
        calm       0.79      0.43      0.56        44
       happy       0.42      0.53      0.47        34
         sad       0.44      0.38      0.41        39
       angry       0.90      0.62      0.73        42
        fear       0.71      0.47      0.57        32
     disgust       0.56      0.44      0.49        32
    surprise       0.66      0.47      0.55        45

    accuracy                           0.50       288
   macro avg       0.59      0.52      0.51       288
weighted avg       0.62      0.50      0.53       288

y_test[10:30]=589      disgust
115     surprise
1223         sad
940         fear
141          sad
1305     disgust
695        angry
756         fear
306         calm
350      disgust
1192    surprise
1023     neutral
1140     neutral
620          sad
819         fear
746          sad
654     surprise
415     s

AS = 55,2% then 50,3%

## Improving with standardization

In [19]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.2, random_state=42)

In [20]:
#standardization

transformer = StandardScaler()
transformer.fit(X1_train)

# saving in a pickle
with open('std_transformer2.pickle', 'wb') as file:
    pickle.dump(transformer, file)
    
# loading from a pickle  
with open('std_transformer2.pickle', 'rb') as file:
    loaded_standardized = pickle.load(file)

X1_train = loaded_standardized.transform(X1_train)
X1_test = loaded_standardized.transform(X1_test)

In [21]:
MLPClassifierModeling(X1_train, X1_test, y1_train, y1_test)

Accuracy of the Recognizer is: 73.3%
              precision    recall  f1-score   support

     neutral       0.71      0.75      0.73        20
        calm       0.86      0.86      0.86        44
       happy       0.53      0.59      0.56        34
         sad       0.72      0.79      0.76        39
       angry       0.82      0.79      0.80        42
        fear       0.70      0.72      0.71        32
     disgust       0.69      0.78      0.74        32
    surprise       0.79      0.58      0.67        45

    accuracy                           0.73       288
   macro avg       0.73      0.73      0.73       288
weighted avg       0.74      0.73      0.73       288

y_test[10:30]=589      disgust
115     surprise
1223         sad
940         fear
141          sad
1305     disgust
695        angry
756         fear
306         calm
350      disgust
1192    surprise
1023     neutral
1140     neutral
620          sad
819         fear
746          sad
654     surprise
415     s

AS = 76,7% Then 73.3%

## Removing neutral

In [22]:
df_features = pd.read_csv('mel_chroma_mfccs_24_8.csv')
df_features.head()

,gender,emotion,actor,0,1,2,3,4,5,6,...,30.1,31.1,32.1,33.1,34.1,35.1,36.1,37.1,38.1,39.1
0,male,neutral,1,0.000011,0.001976,0.015901,0.032733,0.021538,0.008141,0.028234,...,-0.320704,-3.949646,-1.170431,-0.233570,-1.211523,0.016120,-1.144806,-2.061026,-1.967518,-1.471878
1,male,neutral,1,0.000014,0.001849,0.014971,0.047551,0.035372,0.006433,0.024542,...,-1.516942,-4.404281,-0.814472,0.109777,-2.077316,-0.285888,-0.560736,-2.559335,-1.591917,-1.465123
2,male,neutral,1,0.000017,0.001908,0.017518,0.046132,0.024488,0.009170,0.032045,...,-0.307369,-3.435000,-1.965031,-0.523855,-1.918070,-0.327916,-1.371087,-2.072444,-2.138656,-1.685442
3,male,neutral,1,0.000014,0.003084,0.023976,0.059002,0.025616,0.006316,0.024183,...,-0.903333,-3.661112,-1.938478,-0.868744,-1.534840,0.441906,-1.456406,-2.480440,-2.130476,-2.133232
4,male,calm,1,0.000012,0.000789,0.008584,0.023643,0.024978,0.006251,0.016150,...,-1.139909,-4.333735,-0.504630,-0.390943,-2.260401,0.173424,-1.041390,-1.922247,-0.495608,-1.814612


In [23]:
filtered = df_features.loc[
                        (df_features["emotion"] =='calm')|
                        (df_features["emotion"] =='happy')|
                        (df_features["emotion"] =='sad')|
                        (df_features["emotion"] =='angry')|
                        (df_features["emotion"] =='fear')|
                        (df_features["emotion"] =='disgust')|
                        (df_features["emotion"] =='surprise')]
print(filtered)

      gender   emotion  actor         0         1         2         3  \
4       male      calm      1  0.000012  0.000789  0.008584  0.023643   
5       male      calm      1  0.000017  0.003102  0.016791  0.021407   
6       male      calm      1  0.000018  0.004411  0.037020  0.043391   
7       male      calm      1  0.000020  0.004009  0.044063  0.057578   
8       male      calm      1  0.000022  0.005696  0.028871  0.020790   
...      ...       ...    ...       ...       ...       ...       ...   
1435  female  surprise     24  0.000005  0.000044  0.000066  0.000024   
1436  female  surprise     24  0.000080  0.000225  0.000271  0.000161   
1437  female  surprise     24  0.000043  0.000203  0.000576  0.000313   
1438  female  surprise     24  0.002898  0.001060  0.000360  0.000253   
1439  female  surprise     24  0.010942  0.002726  0.000589  0.000361   

             4         5         6  ...      30.1      31.1      32.1  \
4     0.024978  0.006251  0.016150  ... -1.139909 

In [28]:
features_emotions = pd.DataFrame(filtered).reset_index(drop=True)
features_emotions.info()
features_emotions.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1344 entries, 0 to 1343
Columns: 183 entries, gender to 39.1
dtypes: float64(180), int64(1), object(2)
memory usage: 1.9+ MB


,gender,emotion,actor,0,1,2,3,4,5,6,...,30.1,31.1,32.1,33.1,34.1,35.1,36.1,37.1,38.1,39.1
0,male,calm,1,0.000012,0.000789,0.008584,0.023643,0.024978,0.006251,0.016150,...,-1.139909,-4.333735,-0.504630,-0.390943,-2.260401,0.173424,-1.041390,-1.922247,-0.495608,-1.814612
1,male,calm,1,0.000017,0.003102,0.016791,0.021407,0.029746,0.011495,0.012319,...,-1.545056,-3.736922,-1.028613,-0.975852,-1.713234,-0.420986,-2.059035,-2.811184,-1.837945,-2.944513
2,male,calm,1,0.000018,0.004411,0.037020,0.043391,0.007283,0.010791,0.046685,...,-0.802485,-4.664270,-0.597343,-0.737719,-2.822680,-0.238463,-2.313240,-2.056827,-1.520409,-1.835006
3,male,calm,1,0.000020,0.004009,0.044063,0.057578,0.008987,0.011594,0.061087,...,-0.595835,-3.553316,-0.959295,-1.277231,-2.743799,-0.272538,-2.418390,-2.588644,-0.797496,-2.084662
4,male,calm,1,0.000022,0.005696,0.028871,0.020790,0.002965,0.006008,0.017144,...,-0.771233,-3.249421,0.532312,-1.693076,-1.754362,-0.835209,-2.001338,-3.333159,-2.674223,-2.123013


In [25]:
# checking if neutral is not present in the dataframe
result = features_emotions.loc[
                        (features_emotions["emotion"] =='neutral')
                        ]
print(result)

Empty DataFrame
Columns: [gender, emotion, actor, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, ...]
Index: []

[0 rows x 183 columns]


In [38]:
features_emotions.to_csv('mel_chroma_mfccs_24_7.csv',index = False)

In [49]:
features_emotions= pd.read_csv('mel_chroma_mfccs_24_7.csv')
features_emotions.head()

,Unnamed: 0,gender,emotion,actor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,0.1,1.1,2.1,3.1,4.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1,0.2,1.2,2.2,3.2,4.2,5.2,6.2,7.2,8.2,9.2,10.2,11.2,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1,21.1,22.1,23.1,24.1,25.1,26.1,27.1,28.1,29.1,30.1,31.1,32.1,33.1,34.1,35.1,36.1,37.1,38.1,39.1
0,0,male,calm,1,0.000012,0.000789,0.008584,0.023643,0.024978,0.006251,0.016150,0.017182,0.019608,0.018035,0.006866,0.004610,0.010312,0.019593,0.020324,0.006712,0.001820,0.007984,0.009848,0.001885,0.000792,0.003688,0.004912,0.001009,0.000352,0.000334,0.000239,0.000103,0.000127,0.000227,0.000252,0.000423,0.000125,0.000209,0.000392,0.001427,0.001162,0.000189,0.000317,0.000813,0.000229,0.000284,0.000428,0.000135,0.000854,0.000672,0.000174,0.000263,0.000260,0.000224,0.000811,0.000487,0.000085,0.000091,0.000067,0.000084,0.000124,0.000143,0.000159,0.000330,0.000303,0.000175,0.000101,0.000054,0.000023,0.000015,0.000016,0.000022,0.000037,0.000052,0.000075,0.000098,0.000074,0.000029,0.000008,0.000007,0.000016,0.000021,0.000026,0.000028,0.000046,0.000040,0.000026,0.000037,0.000044,0.000061,0.000055,0.000045,0.000037,0.000014,0.000008,0.000007,0.000007,0.000007,0.000006,0.000008,0.000005,0.000003,0.000005,0.000004,0.000006,0.000008,0.000003,0.000004,0.000005,0.000004,0.000003,0.000004,0.000005,0.000003,0.000002,2.158778e-06,1.568582e-06,1.628014e-06,1.118427e-06,9.224078e-07,6.866002e-07,4.452201e-07,4.095236e-07,2.141846e-07,1.430198e-07,9.130326e-08,5.189249e-08,2.195618e-08,4.613697e-09,9.178104e-10,9.985994e-11,5.567851e-12,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-727.783325,87.151093,5.981500,16.521162,7.651807,15.974822,-4.892543,5.531658,-8.175882,-0.301242,-11.974799,-0.529750,0.692740,-1.977111,-0.084285,0.115052,-3.832083,1.656441,2.074690,-4.249748,-1.789677,-0.458033,0.164515,-2.413014,-3.059638,-2.249351,-0.738288,-3.337880,-4.075123,-0.432539,-1.139909,-4.333735,-0.504630,-0.390943,-2.260401,0.173424,-1.041390,-1.922247,-0.495608,-1.814612
1,1,male,calm,1,0.000017,0.003102,0.016791,0.021407,0.029746,0.011495,0.012319,0.006782,0.013946,0.023442,0.009255,0.003194,0.007353,0.013187,0.030339,0.022584,0.006136,0.002648,0.003365,0.009951,0.005240,0.002188,0.002535,0.004221,0.002419,0.000305,0.000177,0.000130,0.000282,0.000397,0.000465,0.000744,0.000528,0.000417,0.000850,0.001355,0.000750,0.000438,0.000448,0.000182,0.000409,0.000865,0.000400,0.000325,0.000404,0.000284,0.000410,0.000225,0.000423,0.000388,0.000360,0.000634,0.000249,0.000061,0.000100,0.000058,0.000085,0.000101,0.000298,0.000244,0.000311,0.000146,0.000091,0.000040,0.000033,0.000017,0.000023,0.000031,0.000051,0.000069,0.000155,0.000118,0.000047,0.000015,0.000005,0.000003,0.000003,0.000010,0.000029,0.000029,0.000025,0.000030,0.000022,0.000020,0.000016,0.000030,0.000039,0.000029,0.000018,0.000013,0.000007,0.000005,0.000003,0.000004,0.000006,0.000005,0.000004,0.000003,0.000004,0.000004,0.000004,0.000004,0.000005,0.000007,0.000009,0.000008,0.000008,0.000006,0.000003,0.000002,0.000002,1.272558e-06,1.126081e-06,1.040018e-06,1.084160e-06,1.362575e-06,1.298073e-06,5.357613e-07,3.364116e-07,3.311025e-07,1.439164e-07,4.063457e-08,1.761115e-08,8.868716e-09,2.765349e-09,5.057543e-10,5.419542e-11,4.823885e-12,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-704.303772,93.277084,9.311788,12.922705,7.454835,17.394232,-4.342582,3.875228,-8.110920,-0.968131,-9.430532,-1.396702,2.581494,-1.416424,0.146414,-0.556509,-3.518421,0.420847,2.156458,-3.789995,-2.339234,-1.558570,1.003638,-2.425308,-2.732662,-2.34

In [50]:
features_emotions= features_emotions.drop('Unnamed: 0', axis=1)
features_emotions.head()

,gender,emotion,actor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,0.1,1.1,2.1,3.1,4.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1,0.2,1.2,2.2,3.2,4.2,5.2,6.2,7.2,8.2,9.2,10.2,11.2,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1,21.1,22.1,23.1,24.1,25.1,26.1,27.1,28.1,29.1,30.1,31.1,32.1,33.1,34.1,35.1,36.1,37.1,38.1,39.1
0,male,calm,1,0.000012,0.000789,0.008584,0.023643,0.024978,0.006251,0.016150,0.017182,0.019608,0.018035,0.006866,0.004610,0.010312,0.019593,0.020324,0.006712,0.001820,0.007984,0.009848,0.001885,0.000792,0.003688,0.004912,0.001009,0.000352,0.000334,0.000239,0.000103,0.000127,0.000227,0.000252,0.000423,0.000125,0.000209,0.000392,0.001427,0.001162,0.000189,0.000317,0.000813,0.000229,0.000284,0.000428,0.000135,0.000854,0.000672,0.000174,0.000263,0.000260,0.000224,0.000811,0.000487,0.000085,0.000091,0.000067,0.000084,0.000124,0.000143,0.000159,0.000330,0.000303,0.000175,0.000101,0.000054,0.000023,0.000015,0.000016,0.000022,0.000037,0.000052,0.000075,0.000098,0.000074,0.000029,0.000008,0.000007,0.000016,0.000021,0.000026,0.000028,0.000046,0.000040,0.000026,0.000037,0.000044,0.000061,0.000055,0.000045,0.000037,0.000014,0.000008,0.000007,0.000007,0.000007,0.000006,0.000008,0.000005,0.000003,0.000005,0.000004,0.000006,0.000008,0.000003,0.000004,0.000005,0.000004,0.000003,0.000004,0.000005,0.000003,0.000002,2.158778e-06,1.568582e-06,1.628014e-06,1.118427e-06,9.224078e-07,6.866002e-07,4.452201e-07,4.095236e-07,2.141846e-07,1.430198e-07,9.130326e-08,5.189249e-08,2.195618e-08,4.613697e-09,9.178104e-10,9.985994e-11,5.567851e-12,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-727.783325,87.151093,5.981500,16.521162,7.651807,15.974822,-4.892543,5.531658,-8.175882,-0.301242,-11.974799,-0.529750,0.692740,-1.977111,-0.084285,0.115052,-3.832083,1.656441,2.074690,-4.249748,-1.789677,-0.458033,0.164515,-2.413014,-3.059638,-2.249351,-0.738288,-3.337880,-4.075123,-0.432539,-1.139909,-4.333735,-0.504630,-0.390943,-2.260401,0.173424,-1.041390,-1.922247,-0.495608,-1.814612
1,male,calm,1,0.000017,0.003102,0.016791,0.021407,0.029746,0.011495,0.012319,0.006782,0.013946,0.023442,0.009255,0.003194,0.007353,0.013187,0.030339,0.022584,0.006136,0.002648,0.003365,0.009951,0.005240,0.002188,0.002535,0.004221,0.002419,0.000305,0.000177,0.000130,0.000282,0.000397,0.000465,0.000744,0.000528,0.000417,0.000850,0.001355,0.000750,0.000438,0.000448,0.000182,0.000409,0.000865,0.000400,0.000325,0.000404,0.000284,0.000410,0.000225,0.000423,0.000388,0.000360,0.000634,0.000249,0.000061,0.000100,0.000058,0.000085,0.000101,0.000298,0.000244,0.000311,0.000146,0.000091,0.000040,0.000033,0.000017,0.000023,0.000031,0.000051,0.000069,0.000155,0.000118,0.000047,0.000015,0.000005,0.000003,0.000003,0.000010,0.000029,0.000029,0.000025,0.000030,0.000022,0.000020,0.000016,0.000030,0.000039,0.000029,0.000018,0.000013,0.000007,0.000005,0.000003,0.000004,0.000006,0.000005,0.000004,0.000003,0.000004,0.000004,0.000004,0.000004,0.000005,0.000007,0.000009,0.000008,0.000008,0.000006,0.000003,0.000002,0.000002,1.272558e-06,1.126081e-06,1.040018e-06,1.084160e-06,1.362575e-06,1.298073e-06,5.357613e-07,3.364116e-07,3.311025e-07,1.439164e-07,4.063457e-08,1.761115e-08,8.868716e-09,2.765349e-09,5.057543e-10,5.419542e-11,4.823885e-12,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-704.303772,93.277084,9.311788,12.922705,7.454835,17.394232,-4.342582,3.875228,-8.110920,-0.968131,-9.430532,-1.396702,2.581494,-1.416424,0.146414,-0.556509,-3.518421,0.420847,2.156458,-3.789995,-2.339234,-1.558570,1.003638,-2.425308,-2.732662,-2.345526,-1.238754,

In [51]:
# drop the target value
X = features_emotions.drop('emotion', axis=1)
y = features_emotions['emotion']

In [52]:
#encoding gender feature

X= pd.get_dummies(X)
X.head()

,actor,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,0.1,1.1,2.1,3.1,4.1,5.1,6.1,7.1,8.1,9.1,10.1,11.1,0.2,1.2,2.2,3.2,4.2,5.2,6.2,7.2,8.2,9.2,10.2,11.2,12.1,13.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1,21.1,22.1,23.1,24.1,25.1,26.1,27.1,28.1,29.1,30.1,31.1,32.1,33.1,34.1,35.1,36.1,37.1,38.1,39.1,gender_female,gender_male
0,1,0.000012,0.000789,0.008584,0.023643,0.024978,0.006251,0.016150,0.017182,0.019608,0.018035,0.006866,0.004610,0.010312,0.019593,0.020324,0.006712,0.001820,0.007984,0.009848,0.001885,0.000792,0.003688,0.004912,0.001009,0.000352,0.000334,0.000239,0.000103,0.000127,0.000227,0.000252,0.000423,0.000125,0.000209,0.000392,0.001427,0.001162,0.000189,0.000317,0.000813,0.000229,0.000284,0.000428,0.000135,0.000854,0.000672,0.000174,0.000263,0.000260,0.000224,0.000811,0.000487,0.000085,0.000091,0.000067,0.000084,0.000124,0.000143,0.000159,0.000330,0.000303,0.000175,0.000101,0.000054,0.000023,0.000015,0.000016,0.000022,0.000037,0.000052,0.000075,0.000098,0.000074,0.000029,0.000008,0.000007,0.000016,0.000021,0.000026,0.000028,0.000046,0.000040,0.000026,0.000037,0.000044,0.000061,0.000055,0.000045,0.000037,0.000014,0.000008,0.000007,0.000007,0.000007,0.000006,0.000008,0.000005,0.000003,0.000005,0.000004,0.000006,0.000008,0.000003,0.000004,0.000005,0.000004,0.000003,0.000004,0.000005,0.000003,0.000002,2.158778e-06,1.568582e-06,1.628014e-06,1.118427e-06,9.224078e-07,6.866002e-07,4.452201e-07,4.095236e-07,2.141846e-07,1.430198e-07,9.130326e-08,5.189249e-08,2.195618e-08,4.613697e-09,9.178104e-10,9.985994e-11,5.567851e-12,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-727.783325,87.151093,5.981500,16.521162,7.651807,15.974822,-4.892543,5.531658,-8.175882,-0.301242,-11.974799,-0.529750,0.692740,-1.977111,-0.084285,0.115052,-3.832083,1.656441,2.074690,-4.249748,-1.789677,-0.458033,0.164515,-2.413014,-3.059638,-2.249351,-0.738288,-3.337880,-4.075123,-0.432539,-1.139909,-4.333735,-0.504630,-0.390943,-2.260401,0.173424,-1.041390,-1.922247,-0.495608,-1.814612,0,1
1,1,0.000017,0.003102,0.016791,0.021407,0.029746,0.011495,0.012319,0.006782,0.013946,0.023442,0.009255,0.003194,0.007353,0.013187,0.030339,0.022584,0.006136,0.002648,0.003365,0.009951,0.005240,0.002188,0.002535,0.004221,0.002419,0.000305,0.000177,0.000130,0.000282,0.000397,0.000465,0.000744,0.000528,0.000417,0.000850,0.001355,0.000750,0.000438,0.000448,0.000182,0.000409,0.000865,0.000400,0.000325,0.000404,0.000284,0.000410,0.000225,0.000423,0.000388,0.000360,0.000634,0.000249,0.000061,0.000100,0.000058,0.000085,0.000101,0.000298,0.000244,0.000311,0.000146,0.000091,0.000040,0.000033,0.000017,0.000023,0.000031,0.000051,0.000069,0.000155,0.000118,0.000047,0.000015,0.000005,0.000003,0.000003,0.000010,0.000029,0.000029,0.000025,0.000030,0.000022,0.000020,0.000016,0.000030,0.000039,0.000029,0.000018,0.000013,0.000007,0.000005,0.000003,0.000004,0.000006,0.000005,0.000004,0.000003,0.000004,0.000004,0.000004,0.000004,0.000005,0.000007,0.000009,0.000008,0.000008,0.000006,0.000003,0.000002,0.000002,1.272558e-06,1.126081e-06,1.040018e-06,1.084160e-06,1.362575e-06,1.298073e-06,5.357613e-07,3.364116e-07,3.311025e-07,1.439164e-07,4.063457e-08,1.761115e-08,8.868716e-09,2.765349e-09,5.057543e-10,5.419542e-11,4.823885e-12,0.767402,0.763228,0.72857,0.724856,0.720314,0.717113,0.749522,0.782215,0.7834,0.762521,0.756099,0.753244,-704.303772,93.277084,9.311788,12.922705,7.454835,17.394232,-4.342582,3.875228,-8.110920,-0.968131,-9.430532,-1.396702,2.581494,-1.416424,0.146414,-0.556509,-3.518421,0.420847,2.156458,-3.789995,-2.339234,-1.558570,1.003638,-2.425308,-2.732662,-2.345526,-1.238754,-2.41

In [53]:
X1 = X.copy()

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [55]:
MLPClassifierModeling(X_train, X_test, y_train, y_test)

Accuracy of the Recognizer is: 55.0%
              precision    recall  f1-score   support

     neutral       0.00      0.00      0.00         0
        calm       0.81      0.57      0.67        37
       happy       0.64      0.33      0.43        43
         sad       0.37      0.84      0.51        44
       angry       0.70      0.66      0.68        35
        fear       0.65      0.45      0.53        29
     disgust       0.49      0.52      0.51        42
    surprise       0.82      0.46      0.59        39

   micro avg       0.55      0.55      0.55       269
   macro avg       0.56      0.48      0.49       269
weighted avg       0.63      0.55      0.55       269

y_test[10:30]=588        angry
175         calm
908        happy
1199         sad
430         fear
43       disgust
353          sad
115         calm
579          sad
374         fear
78           sad
49      surprise
566         calm
451         calm
527          sad
1148       angry
1052     disgust
1046     

AS = 55%

Improving with standardization

In [57]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size=0.2, random_state=42)

In [58]:
#standardization

transformer = StandardScaler()
transformer.fit(X1_train)

# saving in a pickle
with open('std_transformer3.pickle', 'wb') as file:
    pickle.dump(transformer, file)
    
# loading from a pickle  
with open('std_transformer3.pickle', 'rb') as file:
    loaded_standardized = pickle.load(file)

X1_train = loaded_standardized.transform(X1_train)
X1_test = loaded_standardized.transform(X1_test)

In [59]:
MLPClassifierModeling(X1_train, X1_test, y1_train, y1_test)

Accuracy of the Recognizer is: 76.2%
              precision    recall  f1-score   support

     neutral       0.00      0.00      0.00         0
        calm       0.85      0.95      0.90        37
       happy       0.72      0.60      0.66        43
         sad       0.78      0.70      0.74        44
       angry       0.74      0.83      0.78        35
        fear       0.62      0.90      0.73        29
     disgust       0.78      0.69      0.73        42
    surprise       0.85      0.74      0.79        39

   micro avg       0.76      0.76      0.76       269
   macro avg       0.67      0.68      0.67       269
weighted avg       0.77      0.76      0.76       269

y_test[10:30]=588        angry
175         calm
908        happy
1199         sad
430         fear
43       disgust
353          sad
115         calm
579          sad
374         fear
78           sad
49      surprise
566         calm
451         calm
527          sad
1148       angry
1052     disgust
1046     

AS = 76,2%